In [1]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

import numpy as np
import scipy.io

mat = scipy.io.loadmat('pathname/SVHN/Data/train_32x32.mat')
X_train = mat['X']
y_train = mat['y']

mat = scipy.io.loadmat('pathname/SVHN/Data/test_32x32.mat')
X_test = mat['X']
y_test = mat['y']

X_train = np.transpose(X_train, (3, 0, 1, 2)).astype(np.float32) / 255.0
X_test = np.transpose(X_test, (3, 0, 1, 2)).astype(np.float32) / 255.0
y_train = y_train[:,0].astype(np.int32) - 1
y_test = y_test[:,0].astype(np.int32) - 1

#Creating validation sets
val_size = 2000
X_valid, X_train = X_train[:val_size], X_train[val_size:]
y_valid, y_train = y_train[:val_size], y_train[val_size:]

In [2]:
import tensorflow as tf
import numpy as np
from datetime import datetime
import math

def execute_code(rand_seed = 1, lambda_exp = -6.0):
    LOC = """

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

print("Now:", now)

n_hidden1 = 4096
n_hidden2 = 4096
n_outputs = len(np.unique(y_train))

def reset_graph(seed=43):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph(@rand_seed@)

epsVal = pow(10.0, -8.0)

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs + n_neurons)
        init = tf.random_normal((n_inputs, n_neurons), mean=0.0, stddev=stddev)
        
        W = tf.Variable(init, name="kernel")
        #W = tf.cast(tf.abs(W) >= epsVal, W.dtype) * W
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.add(tf.matmul(X, W), b)
                
        if activation is not None:
            return activation(Z)
        else:
            return Z

############

conv1_fmaps = 96
conv1_ksize = 5
conv1_stride = 1
conv1_pad = "SAME"

conv2_fmaps = 128
conv2_ksize = 5
conv2_stride = 1
conv2_pad = "SAME"

conv3_fmaps = 256
conv3_ksize = 5
conv3_stride = 1
conv3_pad = "SAME"

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, 32, 32, 3], name="X")
    y = tf.placeholder(tf.int64, shape=(None), name="y")

conv1 = tf.layers.conv2d(X, filters=conv1_fmaps, kernel_size=conv1_ksize, 
                         strides=conv1_stride, padding=conv1_pad, name="conv1")  
bn1 = tf.layers.batch_normalization(conv1, training=True, momentum=0.9)
bn1_act = tf.nn.relu(bn1)

pool1 = tf.nn.max_pool(bn1_act, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="SAME")

conv2 = tf.layers.conv2d(pool1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad, name="conv2")   
bn2 = tf.layers.batch_normalization(conv2, training=True, momentum=0.9)
bn2_act = tf.nn.relu(bn2)

pool2 = tf.nn.max_pool(bn2_act, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="SAME")

conv3 = tf.layers.conv2d(pool2, filters=conv3_fmaps, kernel_size=conv3_ksize,
                         strides=conv3_stride, padding=conv3_pad, name="conv3")
bn3 = tf.layers.batch_normalization(conv3, training=True, momentum=0.9)
bn3_act = tf.nn.relu(bn3)

pool3 = tf.nn.max_pool(bn3_act, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="SAME")

n_fc1 = np.int64(pool3.shape[1]*pool3.shape[2]*pool3.shape[3])
pool3_flat = tf.reshape(pool3, shape=[-1, n_fc1])

###################

with tf.name_scope("dnn"):
    hidden1 = neuron_layer(pool3_flat, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")

hidden1_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden1/kernel:0")
hidden2_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden2/kernel:0")
logits_W = tf.get_default_graph().get_tensor_by_name("dnn/outputs/kernel:0")

def calcRegValue(W, type=1, eps=0.00001, lambdaValScalar=0.00001):
    if type == 1:
        wMax = 1.0
        gt_eps = tf.minimum(1.0, tf.add(1.0, tf.constant(eps/(wMax-eps))*tf.subtract(tf.abs(W), wMax)))
        #gt_eps = 1.0  
        #lt_eps = tf.constant((1-eps)/eps)*tf.abs(W)
        lt_eps = 0.0
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*gt_eps + tf.cast(tf.abs(W) < eps, W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 2:
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*W
        return tf.reduce_sum(reg_value)
    elif type == 3:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.pow(W, 2))))
    elif type == 4:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.abs(W))))
    elif type == 5:
        return tf.reduce_sum(tf.abs(W))
    elif type == 6:
        return tf.reduce_sum(tf.pow(W, 2))
    elif type == 7:
        gt_eps = 1.0  
        epsMat = tf.constant(eps, shape=W.get_shape())
        lt_eps = 1.0 - tf.pow(epsMat, tf.abs(W))
        reg_value = tf.cast(tf.abs(W) >= np.sqrt(eps), W.dtype)*gt_eps + tf.cast(tf.abs(W) < np.sqrt(eps), W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 8:
        gammaVal = pow(10, 0)
        #gammaVal = tf.sqrt(tf.abs(tf.div(tf.reduce_sum(W), tf.reduce_sum(tf.pow(W, 3)))))
        reg_value =  (2/math.pi)*tf.atan(tf.scalar_mul(gammaVal, tf.abs(W)))
    elif type == 9:
        a = 3.7
        lambdaVal = tf.pow(10.0, @lambda_exp@)   
        aLambda = tf.scalar_mul(a, lambdaVal)
        condMat = tf.multiply(tf.cast(tf.abs(W) > lambdaVal, W.dtype), tf.cast(tf.abs(W) <= aLambda, W.dtype))
        lambdaValSq = tf.pow(lambdaVal, 2)
        absW = tf.abs(W)
        reg_value = tf.cast(absW <= lambdaVal, W.dtype)*tf.scalar_mul(lambdaVal, absW) + \
                    condMat*tf.scalar_mul(-0.5/(a-1), tf.pow(W, 2) - 2*tf.scalar_mul(aLambda, absW) + lambdaValSq) + \
                    tf.cast(absW > aLambda, W.dtype)*tf.scalar_mul(0.5*(a+1), lambdaValSq)
        return tf.reduce_sum(reg_value)
    elif type == 10:
        b = 5.0
        lambdaVal = tf.constant(lambdaValScalar, W.dtype)
        absW = tf.abs(W)
        reg_value = tf.cast(absW > b*lambdaVal, W.dtype)*tf.scalar_mul(0.5*b, tf.pow(lambdaVal, 2)) + \
                    tf.cast(absW <= b*lambdaVal, W.dtype)*tf.scalar_mul(lambdaVal, absW - \
                                                                       tf.scalar_mul(1/(2*b*lambdaVal), tf.pow(W, 2))) 
        return tf.reduce_sum(reg_value)
    elif type == 11:
        eMat = tf.constant(math.e, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(eMat, tf.subtract(0.0, 10.0*tf.pow(W, 2)))))
        
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    regType = 11
    lambdaValScalar = pow(10.0, @lambda_exp@)
    total_regvalue = tf.add_n([calcRegValue(hidden1_W, type=regType, eps=epsVal, lambdaValScalar=lambdaValScalar), \
                               calcRegValue(hidden2_W, type=regType, eps=epsVal, lambdaValScalar=lambdaValScalar), \
                               calcRegValue(logits_W, type=regType, eps=epsVal, lambdaValScalar=lambdaValScalar)])
    
    total_nzw = tf.cast(tf.add_n([tf.count_nonzero(hidden1_W), \
                                  tf.count_nonzero(hidden2_W), \
                                  tf.count_nonzero(logits_W)]), tf.float32)
                                  
    total_nzw_eps = tf.cast(tf.add_n([tf.reduce_sum(tf.cast(abs(hidden1_W) < epsVal, hidden1_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden2_W) < epsVal, hidden2_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(logits_W) < epsVal, logits_W.dtype))]), tf.int64)
    
    lambdaParam = tf.pow(10.0, @lambda_exp@, name="lambdaParam")
    loss_total = tf.add(tf.reduce_mean(xentropy, name="loss_total"), tf.scalar_mul(lambdaParam, total_regvalue))
    #loss_total = tf.reduce_mean(xentropy, name="loss_total")
    

    
learning_rate = tf.placeholder(tf.float32, shape=(), name="learning_rate")
with tf.name_scope("train"):
    #optimizer = tf.train.ProximalGradientDescentOptimizer(learning_rate)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    #optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    #optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    training_op = optimizer.minimize(loss_total)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 250
batch_size = 20

def shuffle_batch(X, y, batch_size):
    np.random.seed(43)
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    batch_num = -1
    for batch_idx in np.array_split(rnd_idx, n_batches):
        batch_num = batch_num + 1
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield batch_num, X_batch, y_batch

def get_lr_step(epochCounter, start_lr, stepsize, end_lr, lr_iter_period):
    lr = start_lr - math.floor(epochCounter/lr_iter_period) * stepsize
    return max(lr, end_lr)
        
def get_lr_triangle(epochCounter, lr_min, lr_max, lr_iter_period):
    cycle = math.floor(1 + epochCounter/(2*lr_iter_period))
    x = abs(epochCounter/lr_iter_period - 2*cycle + 1)
    lr = lr_min + (lr_max - lr_min) * max(0, (1-x)/math.pow(2, cycle-1))
    return lr

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

start_time = datetime.now()

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        lr = get_lr_triangle(epoch, lr_min=0.01, lr_max=0.25, lr_iter_period=20)
        #lr=0.01
        rnd_idx = np.random.permutation(len(X_train))
        for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
            X_batch, y_batch = X_train[rnd_indices,:,:,:], y_train[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, learning_rate: lr})
        
        loss_val, acc_val = sess.run([loss_total, accuracy], \
                                                   feed_dict={X: X_valid, y: y_valid, learning_rate: lr})
        
        if best_loss > loss_val:
            save_path = saver.save(sess, "./models/model_" + now + ".ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        
        print("{}\tValidation losses: {:.4f}\tBest Loss: {:.4f} ({})\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, checks_without_progress, acc_val * 100))

end_time = datetime.now()
running_time = end_time - start_time
print("Total running time:\t", running_time.seconds)

with tf.Session() as sess:
    saver.restore(sess, "./models/model_" + now + ".ckpt") # or better, use save_path
    #saver.restore(sess, "./models/model_20200208153222.ckpt") # or better, use save_path
    n_batches = len(X_test) // 100
    batch_remaining = n_batches
    correct_count = 0.0
    for batch_idx in np.array_split(range(len(X_test)), n_batches):
        correct_count = correct_count + len(batch_idx)*accuracy.eval(feed_dict={X: X_test[batch_idx], y: y_test[batch_idx]})
        #print("Batches remaining:    ", batch_remaining)
        batch_remaining = batch_remaining - 1
print("Test error rate:    {:.10f}%".format(100*(1-correct_count/len(X_test)))) 
"""
    LOC = LOC.replace("@rand_seed@", str(rand_seed))
    LOC = LOC.replace("@lambda_exp@", str(lambda_exp))
    exec(LOC)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/

In [ ]:
for j in [-7.0, -6.8, -6.6, -6.4, -6.2, -6.0, -5.8, -5.6, -4.8]:
    for i in [1,2,3]:
        print("RANDOM SEED:  ", i, "\tLAMBDA_EXP:  ", j)
        execute_code(rand_seed=i, lambda_exp=j)

RANDOM SEED:   1 	LAMBDA_EXP:   -7.0
Now: 20211007191018
0	Validation losses: 0.4189	Best Loss: 0.4189 (0)	Accuracy: 87.45%
1	Validation losses: 0.3138	Best Loss: 0.3138 (0)	Accuracy: 90.65%
2	Validation losses: 0.3123	Best Loss: 0.3123 (0)	Accuracy: 90.45%
3	Validation losses: 0.3195	Best Loss: 0.3123 (1)	Accuracy: 91.30%
4	Validation losses: 0.2652	Best Loss: 0.2652 (0)	Accuracy: 92.25%
5	Validation losses: 0.2722	Best Loss: 0.2652 (1)	Accuracy: 92.40%
6	Validation losses: 0.2277	Best Loss: 0.2277 (0)	Accuracy: 94.05%
7	Validation losses: 0.2713	Best Loss: 0.2277 (1)	Accuracy: 92.15%
8	Validation losses: 0.2479	Best Loss: 0.2277 (2)	Accuracy: 93.65%
9	Validation losses: 0.2837	Best Loss: 0.2277 (3)	Accuracy: 93.15%
10	Validation losses: 0.2574	Best Loss: 0.2277 (4)	Accuracy: 93.65%
11	Validation losses: 0.3097	Best Loss: 0.2277 (5)	Accuracy: 92.60%
12	Validation losses: 0.2896	Best Loss: 0.2277 (6)	Accuracy: 93.05%
13	Validation losses: 0.2604	Best Loss: 0.2277 (7)	Accuracy: 94.50%
1

Early stopping!
Total running time:	 38148
INFO:tensorflow:Restoring parameters from ./models/model_20211009035800.ckpt
Test error rate:    7.0912719002%
RANDOM SEED:   2 	LAMBDA_EXP:   -6.8
Now: 20211009143446
0	Validation losses: 0.4288	Best Loss: 0.4288 (0)	Accuracy: 87.60%
1	Validation losses: 0.3269	Best Loss: 0.3269 (0)	Accuracy: 90.85%
2	Validation losses: 0.3341	Best Loss: 0.3269 (1)	Accuracy: 89.70%
3	Validation losses: 0.2915	Best Loss: 0.2915 (0)	Accuracy: 92.25%
4	Validation losses: 0.2890	Best Loss: 0.2890 (0)	Accuracy: 91.90%
5	Validation losses: 0.2909	Best Loss: 0.2890 (1)	Accuracy: 92.25%
6	Validation losses: 0.2959	Best Loss: 0.2890 (2)	Accuracy: 92.75%
7	Validation losses: 0.2390	Best Loss: 0.2390 (0)	Accuracy: 93.55%
8	Validation losses: 0.2453	Best Loss: 0.2390 (1)	Accuracy: 94.05%
9	Validation losses: 0.2606	Best Loss: 0.2390 (2)	Accuracy: 94.20%
10	Validation losses: 0.2637	Best Loss: 0.2390 (3)	Accuracy: 93.55%
11	Validation losses: 0.3098	Best Loss: 0.2390 (4)	

28	Validation losses: 0.4736	Best Loss: 0.2661 (19)	Accuracy: 94.60%
29	Validation losses: 0.4915	Best Loss: 0.2661 (20)	Accuracy: 95.15%
Early stopping!
Total running time:	 42577
INFO:tensorflow:Restoring parameters from ./models/model_20211010224217.ckpt
Test error rate:    6.5342651677%
RANDOM SEED:   3 	LAMBDA_EXP:   -6.6
Now: 20211011103251
0	Validation losses: 0.4378	Best Loss: 0.4378 (0)	Accuracy: 88.05%
1	Validation losses: 0.3911	Best Loss: 0.3911 (0)	Accuracy: 89.40%
2	Validation losses: 0.3205	Best Loss: 0.3205 (0)	Accuracy: 91.45%
3	Validation losses: 0.3236	Best Loss: 0.3205 (1)	Accuracy: 91.65%
4	Validation losses: 0.2826	Best Loss: 0.2826 (0)	Accuracy: 92.60%
5	Validation losses: 0.2782	Best Loss: 0.2782 (0)	Accuracy: 92.55%
6	Validation losses: 0.2806	Best Loss: 0.2782 (1)	Accuracy: 93.10%
7	Validation losses: 0.2699	Best Loss: 0.2699 (0)	Accuracy: 93.25%
8	Validation losses: 0.2740	Best Loss: 0.2699 (1)	Accuracy: 93.70%
9	Validation losses: 0.2993	Best Loss: 0.2699 (2

23	Validation losses: 0.4123	Best Loss: 0.2813 (18)	Accuracy: 94.00%
24	Validation losses: 0.4208	Best Loss: 0.2813 (19)	Accuracy: 94.15%
25	Validation losses: 0.4705	Best Loss: 0.2813 (20)	Accuracy: 93.70%
Early stopping!
Total running time:	 37006
INFO:tensorflow:Restoring parameters from ./models/model_20211012194546.ckpt
Test error rate:    7.5291945720%
RANDOM SEED:   1 	LAMBDA_EXP:   -6.2
Now: 20211013060331
0	Validation losses: 0.5107	Best Loss: 0.5107 (0)	Accuracy: 86.90%
1	Validation losses: 0.4538	Best Loss: 0.4538 (0)	Accuracy: 88.70%
2	Validation losses: 0.4280	Best Loss: 0.4280 (0)	Accuracy: 89.65%
3	Validation losses: 0.3755	Best Loss: 0.3755 (0)	Accuracy: 91.15%
4	Validation losses: 0.3514	Best Loss: 0.3514 (0)	Accuracy: 92.65%
5	Validation losses: 0.3367	Best Loss: 0.3367 (0)	Accuracy: 93.10%
6	Validation losses: 0.3272	Best Loss: 0.3272 (0)	Accuracy: 93.15%
7	Validation losses: 0.3346	Best Loss: 0.3272 (1)	Accuracy: 93.60%
8	Validation losses: 0.3306	Best Loss: 0.3272 

22	Validation losses: 0.4721	Best Loss: 0.3661 (12)	Accuracy: 93.00%
23	Validation losses: 0.4953	Best Loss: 0.3661 (13)	Accuracy: 94.50%
24	Validation losses: 0.5469	Best Loss: 0.3661 (14)	Accuracy: 93.90%
25	Validation losses: 0.5233	Best Loss: 0.3661 (15)	Accuracy: 94.15%
26	Validation losses: 0.4826	Best Loss: 0.3661 (16)	Accuracy: 95.05%
27	Validation losses: 0.5362	Best Loss: 0.3661 (17)	Accuracy: 95.00%
28	Validation losses: 0.5187	Best Loss: 0.3661 (18)	Accuracy: 94.90%
29	Validation losses: 0.5496	Best Loss: 0.3661 (19)	Accuracy: 94.95%
30	Validation losses: 0.5287	Best Loss: 0.3661 (20)	Accuracy: 95.00%
Early stopping!
Total running time:	 45395
INFO:tensorflow:Restoring parameters from ./models/model_20211014154246.ckpt
Test error rate:    6.3921322786%
RANDOM SEED:   2 	LAMBDA_EXP:   -6.0
Now: 20211015042020
0	Validation losses: 0.5371	Best Loss: 0.5371 (0)	Accuracy: 88.20%
1	Validation losses: 0.4560	Best Loss: 0.4560 (0)	Accuracy: 91.35%
2	Validation losses: 0.4886	Best L

...

170	Validation losses: 0.3008	Best Loss: 0.2974 (18)	Accuracy: 94.60%
171	Validation losses: 0.2990	Best Loss: 0.2974 (19)	Accuracy: 94.55%
172	Validation losses: 0.2973	Best Loss: 0.2973 (0)	Accuracy: 94.65%
173	Validation losses: 0.2997	Best Loss: 0.2973 (1)	Accuracy: 94.60%
174	Validation losses: 0.2995	Best Loss: 0.2973 (2)	Accuracy: 94.60%
175	Validation losses: 0.2991	Best Loss: 0.2973 (3)	Accuracy: 94.65%
176	Validation losses: 0.2998	Best Loss: 0.2973 (4)	Accuracy: 94.55%
177	Validation losses: 0.2985	Best Loss: 0.2973 (5)	Accuracy: 94.60%
178	Validation losses: 0.2974	Best Loss: 0.2973 (6)	Accuracy: 94.60%
179	Validation losses: 0.2988	Best Loss: 0.2973 (7)	Accuracy: 94.55%
180	Validation losses: 0.3022	Best Loss: 0.2973 (8)	Accuracy: 94.55%
181	Validation losses: 0.2994	Best Loss: 0.2973 (9)	Accuracy: 94.60%
182	Validation losses: 0.3011	Best Loss: 0.2973 (10)	Accuracy: 94.50%
183	Validation losses: 0.2977	Best Loss: 0.2973 (11)	Accuracy: 94.55%
184	Validation losses: 0.2976	

61	Validation losses: 0.3417	Best Loss: 0.3417 (0)	Accuracy: 94.90%
62	Validation losses: 0.3453	Best Loss: 0.3417 (1)	Accuracy: 94.70%
63	Validation losses: 0.3379	Best Loss: 0.3379 (0)	Accuracy: 94.70%
64	Validation losses: 0.3371	Best Loss: 0.3371 (0)	Accuracy: 94.65%
65	Validation losses: 0.3350	Best Loss: 0.3350 (0)	Accuracy: 94.65%
66	Validation losses: 0.3289	Best Loss: 0.3289 (0)	Accuracy: 94.65%
67	Validation losses: 0.3266	Best Loss: 0.3266 (0)	Accuracy: 94.70%
68	Validation losses: 0.3262	Best Loss: 0.3262 (0)	Accuracy: 94.75%
69	Validation losses: 0.3258	Best Loss: 0.3258 (0)	Accuracy: 94.75%
70	Validation losses: 0.3218	Best Loss: 0.3218 (0)	Accuracy: 94.75%
71	Validation losses: 0.3205	Best Loss: 0.3205 (0)	Accuracy: 94.80%
72	Validation losses: 0.3196	Best Loss: 0.3196 (0)	Accuracy: 94.75%
73	Validation losses: 0.3221	Best Loss: 0.3196 (1)	Accuracy: 94.65%
74	Validation losses: 0.3192	Best Loss: 0.3192 (0)	Accuracy: 94.65%
75	Validation losses: 0.3164	Best Loss: 0.3164 (

181	Validation losses: 0.2737	Best Loss: 0.2733 (8)	Accuracy: 94.70%
182	Validation losses: 0.2723	Best Loss: 0.2723 (0)	Accuracy: 94.65%
183	Validation losses: 0.2718	Best Loss: 0.2718 (0)	Accuracy: 94.70%
184	Validation losses: 0.2722	Best Loss: 0.2718 (1)	Accuracy: 94.65%
185	Validation losses: 0.2729	Best Loss: 0.2718 (2)	Accuracy: 94.65%
186	Validation losses: 0.2721	Best Loss: 0.2718 (3)	Accuracy: 94.70%
187	Validation losses: 0.2724	Best Loss: 0.2718 (4)	Accuracy: 94.65%
188	Validation losses: 0.2719	Best Loss: 0.2718 (5)	Accuracy: 94.60%
189	Validation losses: 0.2715	Best Loss: 0.2715 (0)	Accuracy: 94.70%
190	Validation losses: 0.2728	Best Loss: 0.2715 (1)	Accuracy: 94.70%
191	Validation losses: 0.2725	Best Loss: 0.2715 (2)	Accuracy: 94.70%
192	Validation losses: 0.2725	Best Loss: 0.2715 (3)	Accuracy: 94.75%
193	Validation losses: 0.2731	Best Loss: 0.2715 (4)	Accuracy: 94.70%
194	Validation losses: 0.2724	Best Loss: 0.2715 (5)	Accuracy: 94.70%
195	Validation losses: 0.2727	Best

49	Validation losses: 0.2638	Best Loss: 0.2596 (8)	Accuracy: 94.25%
50	Validation losses: 0.2726	Best Loss: 0.2596 (9)	Accuracy: 94.60%
51	Validation losses: 0.2269	Best Loss: 0.2269 (0)	Accuracy: 94.30%
52	Validation losses: 0.2967	Best Loss: 0.2269 (1)	Accuracy: 94.35%
53	Validation losses: 0.3321	Best Loss: 0.2269 (2)	Accuracy: 93.95%
54	Validation losses: 0.2875	Best Loss: 0.2269 (3)	Accuracy: 94.70%
55	Validation losses: 0.2810	Best Loss: 0.2269 (4)	Accuracy: 94.25%
56	Validation losses: 0.2911	Best Loss: 0.2269 (5)	Accuracy: 94.55%
57	Validation losses: 0.3212	Best Loss: 0.2269 (6)	Accuracy: 94.45%
58	Validation losses: 0.3067	Best Loss: 0.2269 (7)	Accuracy: 94.55%
59	Validation losses: 0.3183	Best Loss: 0.2269 (8)	Accuracy: 94.65%
60	Validation losses: 0.3364	Best Loss: 0.2269 (9)	Accuracy: 93.90%
61	Validation losses: 0.3409	Best Loss: 0.2269 (10)	Accuracy: 94.25%
62	Validation losses: 0.3266	Best Loss: 0.2269 (11)	Accuracy: 94.45%
63	Validation losses: 0.3145	Best Loss: 0.2269

28	Validation losses: 0.3117	Best Loss: 0.3117 (0)	Accuracy: 94.50%
29	Validation losses: 0.2939	Best Loss: 0.2939 (0)	Accuracy: 94.60%
30	Validation losses: 0.3108	Best Loss: 0.2939 (1)	Accuracy: 94.20%
31	Validation losses: 0.2709	Best Loss: 0.2709 (0)	Accuracy: 94.85%
32	Validation losses: 0.2648	Best Loss: 0.2648 (0)	Accuracy: 94.85%
33	Validation losses: 0.2603	Best Loss: 0.2603 (0)	Accuracy: 94.50%
34	Validation losses: 0.2510	Best Loss: 0.2510 (0)	Accuracy: 94.90%
35	Validation losses: 0.2554	Best Loss: 0.2510 (1)	Accuracy: 94.45%
36	Validation losses: 0.2563	Best Loss: 0.2510 (2)	Accuracy: 94.55%
37	Validation losses: 0.2515	Best Loss: 0.2510 (3)	Accuracy: 94.70%
38	Validation losses: 0.2494	Best Loss: 0.2494 (0)	Accuracy: 94.60%
39	Validation losses: 0.2483	Best Loss: 0.2483 (0)	Accuracy: 94.75%
40	Validation losses: 0.2488	Best Loss: 0.2483 (1)	Accuracy: 94.70%
41	Validation losses: 0.2505	Best Loss: 0.2483 (2)	Accuracy: 94.80%
42	Validation losses: 0.2481	Best Loss: 0.2481 (